# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import os

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword = "夏威夷蓋飯"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

夏威夷蓋飯: %E5%A4%8F%E5%A8%81%E5%A4%B7%E8%93%8B%E9%A3%AF
/wiki/%E5%A4%8F%E5%A8%81%E5%A4%B7%E8%93%8B%E9%A3%AF


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [3]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


夏威夷蓋飯（夏威夷語：Poke，音「剖剋」，意為「切塊」）是夏威夷人的一種料理生魚的方法，將生魚切塊，然後用醬汁醃製後蓋於米飯上[3]。

傳統上，夏威夷蓋飯在夏威夷作為一種涼菜，主要食材為小珊瑚魚和章魚，以海鹽醃製好後輔以石栗和黎姆海藻。

隨著日本移民的到來，日本料理對原本的夏威夷蓋飯從用料到形式產生了極大的改變。除原有的食材外，金槍魚更為流行[4]，還可加有毛豆、日式海藻，調味料則變為醬油、小蔥以及香油和柚子醋，並以蓋飯的形式佐壽司薑作為主菜出售。

在傳到美國西岸後，主料則可選為三文魚，當地流行的牛油果、以及墨西哥辣椒和玉米也會被當做食材，而醬料則包括是拉差香甜辣椒醬甚至芥末蛋黃醬[5]



### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [4]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [夏威夷語] /wiki/%E5%A4%8F%E5%A8%81%E5%A4%B7%E8%AF%AD
外部連結: [夏威夷] /wiki/%E5%A4%8F%E5%A8%81%E5%A4%B7
外部連結: [石栗] /wiki/%E7%9F%B3%E6%A0%97
外部連結: [日本料理] /wiki/%E6%97%A5%E6%9C%AC%E6%96%99%E7%90%86
外部連結: [金槍魚] /wiki/%E9%87%91%E6%9E%AA%E9%B1%BC
外部連結: [毛豆] /wiki/%E6%AF%9B%E8%B1%86
外部連結: [柚子醋] /wiki/%E6%9F%9A%E5%AD%90%E9%86%8B
外部連結: [蓋飯] /wiki/%E8%93%8B%E9%A3%AF
外部連結: [美國西岸] /wiki/%E7%BE%8E%E5%9B%BD%E8%A5%BF%E5%B2%B8
外部連結: [三文魚] /wiki/%E4%B8%89%E6%96%87%E9%AD%9A
外部連結: [牛油果] /wiki/%E9%B1%B7%E6%A2%A8
外部連結: [墨西哥辣椒] /wiki/%E5%A2%A8%E8%A5%BF%E5%93%A5%E8%BE%A3%E6%A4%92
外部連結: [玉米] /wiki/%E7%8E%89%E7%B1%B3
外部連結: [是拉差香甜辣椒醬] /wiki/%E6%98%AF%E6%8B%89%E5%B7%AE%E9%A6%99%E7%94%9C%E8%BE%A3%E6%A4%92%E9%86%AC


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [5]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

        #
        # 解析回傳資料，萃取文章內容並儲存

        filename = 'save_content.txt'

        if os.path.exists(filename):
            append_write = 'a' # append if already exists
        else:
            append_write = 'w' # make a new file if not

        for paragraph in content:
            with open(filename, append_write ,encoding='utf-8') as f:
                f.write(paragraph.get_text())   
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})

        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_link] = a_keyword

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [7]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E5%A4%8F%E5%A8%81%E5%A4%B7%E8%93%8B%E9%A3%AF (夏威夷蓋飯)
遞迴層[1] - /wiki/%E5%A4%8F%E5%A8%81%E5%A4%B7%E8%AF%AD (夏威夷語)
遞迴層[1] - /wiki/%E5%A4%8F%E5%A8%81%E5%A4%B7 (夏威夷)
遞迴層[1] - /wiki/%E7%9F%B3%E6%A0%97 (石栗)
遞迴層[1] - /wiki/%E6%97%A5%E6%9C%AC%E6%96%99%E7%90%86 (日本料理)
遞迴層[1] - /wiki/%E9%87%91%E6%9E%AA%E9%B1%BC (金槍魚)
遞迴層[1] - /wiki/%E6%AF%9B%E8%B1%86 (毛豆)
遞迴層[1] - /wiki/%E6%9F%9A%E5%AD%90%E9%86%8B (柚子醋)
遞迴層[1] - /wiki/%E8%93%8B%E9%A3%AF (蓋飯)
遞迴層[1] - /wiki/%E7%BE%8E%E5%9B%BD%E8%A5%BF%E5%B2%B8 (美國西岸)
遞迴層[1] - /wiki/%E4%B8%89%E6%96%87%E9%AD%9A (三文魚)
遞迴層[1] - /wiki/%E9%B1%B7%E6%A2%A8 (牛油果)
遞迴層[1] - /wiki/%E5%A2%A8%E8%A5%BF%E5%93%A5%E8%BE%A3%E6%A4%92 (墨西哥辣椒)
遞迴層[1] - /wiki/%E7%8E%89%E7%B1%B3 (玉米)
遞迴層[1] - /wiki/%E6%98%AF%E6%8B%89%E5%B7%AE%E9%A6%99%E7%94%9C%E8%BE%A3%E6%A4%92%E9%86%AC (是拉差香甜辣椒醬)
